In [78]:
from tensorflow import keras # Need this when working in a Jupyter notebook

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


from keras import backend as K
from keras.models import Sequential # Model used to create the architecture both ANNs and CNNs
from keras.layers.core import Dense 
# from keras.optimizers import SGD 
from tensorflow.keras.optimizers import SGD # Use the above instead in GCP. Colab can use the previous line.
from keras.layers.convolutional import Conv2D # For CNNs
from keras.layers.convolutional import MaxPooling2D # For CNNs
from keras.layers.core import Flatten # For CNNs
from keras.layers.core import Activation # For CNNs
from keras.layers.core import Dropout # For CNNs

import matplotlib.pyplot as plt
import glob
import os as os

import numpy as np
import itertools

from scipy import ndimage
from matplotlib.colors import LogNorm

Bucket mount instruction in image preparation notebook

In [79]:
dataDir = 'gcs/resize'

In [80]:
cd ~/

/home/jupyter


In [81]:
objects = os.listdir(dataDir)
objects = objects[1:]
objects

['dragon_snake', 'flower', 'fox_tiger', 'mix', 'small_animal']

# Binarizing the labels

In [82]:
dragon_snake_files = globFn(objects[0])
flower_files = globFn(objects[1])
mix_files = globFn(objects[2])
fox_tiger_files = globFn(objects[4])

In [83]:
stackedArrayLabels = []
stackedArrayLabels.append(objects[1])
print(stackedArrayLabels)

lb = LabelBinarizer()
binarizedLabels = lb.fit_transform(stackedArrayLabels)
print(binarizedLabels)
print(objects)

['flower']
[[0]]
['dragon_snake', 'flower', 'fox_tiger', 'mix', 'small_animal']


The binarized label doesn't work as intended because there's only one image to binarize it's label. If all images were included in the binarization then the binarized labels should look like 

$\big[$0, 1, 0, 0, 0$\big]$

# ANN model

Can put the following cell into a function based on data augmentation parameters. First have to learn how to use the data augmentation function from the Keras package. 

## normalising and flattening the images

In [84]:
ls

IG-Trends-with-Machine-Learning/  gcs/  src/  tutorials/


In [85]:
dataDir

'gcs/resize'

In [86]:
stackedArrayValues = [] # Normalised, resized and stacked 1D arrays corresponding to the images.
stackedArrayLabels = [] # 1D array for the labels.

# Finding all the images in each type that starts the object type.
for imageType in objects:
    image_type_folder = os.path.abspath(dataDir + '/' + imageType)
    files = glob.glob(image_type_folder + '/' + imageType + '_*.jpg')

# For each source, we want to resize and normalise the image.
    for filename in files:
        data = plt.imread(filename)
        # resizeData = ndimage.zoom(data,zoom=zoom_factor) 

        # Normalisation is done by first subtracting the minimum value, and then dividing by the maximum value in the subtracted values.
        normalisedData = (data-np.min(data))/np.max(data-np.min(data))
        normalised1D = normalisedData.flatten() # Turning the 2D image into a 1D array.
        stackedArrayValues.append(normalised1D)
        stackedArrayLabels.append(imageType)
        
# Converting the lists into a 1D array.
stackedArrayValues = np.array(stackedArrayValues)
stackedArrayLabels = np.array(stackedArrayLabels)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [145]:
stackedArrayValues = [] # Normalised, resized and stacked 1D arrays corresponding to the images.
stackedArrayLabels = [] # 1D array for the labels.

# Finding all the images in each type that starts the object type.
for imageType in objects:
    image_type_folder = os.path.abspath(dataDir + '/' + imageType)
    files = glob.glob(image_type_folder + '/' + imageType + '_*.jpg')

# For each source, we want to resize and normalise the image.
    for filename in files:
        arrayValues = np.array([])
        arrayLabels = np.array([])
        data = plt.imread(filename)
        # resizeData = ndimage.zoom(data,zoom=zoom_factor) 

        # Normalisation is done by first subtracting the minimum value, and then dividing by the maximum value in the subtracted values.
        normalisedData = (data-np.min(data))/np.max(data-np.min(data))
        normalised1D = normalisedData.flatten() # Turning the 2D image into a 1D array.
        
        # arrayValues.append(normalised1D)
        arrayValues = np.append(arrayValues, normalised1D)
        


        
        # When appending to the stacked array, we want to make sure the flattened images have the same dimensions
        if arrayValues.shape[0] == 150528:
            stackedArrayValues.append(arrayValues)
            
            # Appending the image types
            stackedArrayLabels.append(imageType)
        
        
        
# Converting the lists into a 1D array.
stackedArrayValues = np.array(stackedArrayValues)
stackedArrayLabels = np.array(stackedArrayLabels)

In [141]:
stackedArrayValues.shape

(388, 150528)

In [146]:
stackedArrayLabels.shape

(388,)

### Checking if there are flat images that are different in shapes. 

This has been corrected in the stacked array appending cell above. The flat images of different shapes have simply been excluded. Need to learn why they have different shapes later. 

Finding the location of the flat images that are different. 

Removing the flat images with different shapes.

## Binarising the labels

In [147]:
# Binarising the labels once all the sources for each object were identified, resized and normalised:
lb = LabelBinarizer()
binarizedLabels = lb.fit_transform(stackedArrayLabels)

In [148]:
print(lb.classes_)

['dragon_snake' 'flower' 'fox_tiger' 'mix' 'small_animal']


## Splitting the data

In [149]:
trainData, testData, trainLabels, testLabels = train_test_split(stackedArrayValues, binarizedLabels, test_size=0.3)

In [150]:
print(f"Shape of trainData: {trainData.shape}")
print(f"Shape of testData: {testData.shape}")

Shape of trainData: (271, 150528)
Shape of testData: (117, 150528)


## Running the ANN

In [152]:
model = Sequential() #Sequential is the model, so defining it in a variable so we can use it later.

model.add(Dense(256, input_shape=(150528,), activation="sigmoid"))
#Sigmoid activates the layer.
#first parameter of Dense() is the nodes, we used these numbers in the lectorial. Second parameter is the shape of the image

model.add(Dense(128, activation="sigmoid"))#hidden layer

model.add(Dense(3, activation="softmax"))#first parameter is how many classifications we have. 
#we have three layers, one input layer, one hidden layer and one output layer. 


# ----------------------------------------

learnRate = 0.01 #one of the most important parameters to set in the model. 
#learnRate is for determining how much to change a weight to drive the loss function to a minimum. 
#higher it's set = faster the model runs, but less accurate it is. 

opt = SGD(learnRate,momentum=0.9) #used for optimising the model. 

# ----------------------------------------

model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])#compiling the model

epochs = 100 #number of runs the model does. The more epochs can cause overfit and too little epochs will be underfit. 

#applying the data to the model. 
H = model.fit(trainData, trainLabels, validation_data=(testData, testLabels),
              epochs=epochs, batch_size=64, verbose = 0 )


#applying the model to all the test data to check how well it's performed. 
predictionsANN = model.predict(testData)

# Print a formatted report
print(classification_report(testLabels.argmax(axis=1), 
                                                  
                            predictionsANN.argmax(axis=1), 
                            target_names=[str(x) for x in lb.classes_]))#sets the labels for each output
                                                                        



ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 919, in compute_loss
        y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
  

# CNN model

for cnn have to consider the astro objects had 3 different wavelenghts for each image

In [ ]:
def cnnPrepFn(directory, source_type, rotation_factor):
  # Similar initial process as the ANN function:
  files = glob.glob(directory + '/' + source_type+'/'+'*_I4.fits')
  # The amount of zooming for the images will impact the image's dimensions.
  # Since 302x302 is the original size of the images, we want the dimension of the zoomed-in images which depends on the zoom factor.
  imagesArr = np.zeros((100,int(zoom_factor*302),int(zoom_factor*302),3)) 
  labelsList = [] # The labels will become a 1D array.

  # Identifying the 3 images that will be resized and normalised:
  for indx in np.arange(0,len(files)):
    source = files[indx].split('_I4.fits')[0] # Finding the name for each source by splitting the string from `glob.glob` into two parts.
                                              # We don't want the text starting from _, so we use [0] to get the source name only.
    i2Image = plt.imread(source+'_I2.fits') # The I2 image for the source.

    # Resizing the three Spitzer-band images above and finding the maximum value in each image:
    resizeI2 = ndimage.rotate(i2Image, rotation_factor, reshape = False)
    resizeI2Max = np.max(resizeI2)


    # Normalisation is done by finding the greatest value in the maxima for each of the 3 images.
    # This is so all the values are smaller than or equal to 1 (for the CNN), and maintain the colour differences between them.
    normFactor = np.max([resizeI2Max, resizeI3Max, resizeI4Max])

    # Applying the normalisation factor to the resized images:
    normI2 = resizeI2/normFactor

    # Adding the resized and normalised images into the iamge array:
    imagesArr[indx,:,:,0] = normI2

    # Adding the label that identifies the source type (HII, PNE, RG):
    labelsList.append(source_type)

  # Converting the list of labels into an array:
  labelsArr = np.array(labelsList)

  return imagesArr, labelsArr

In [ ]:
HII_Images_0, HII_Labels_0 = cnnPrepFn2(dir, "HII", zoom_factor=0.6, rotation_factor= 0)
PNE_Images_0, PNE_Labels_0 = cnnPrepFn2(dir, "PNE", zoom_factor=0.6, rotation_factor= 0)
RG_Images_0, RG_Labels_0 = cnnPrepFn2(dir, "RG", zoom_factor=0.6, rotation_factor= 0)

In [ ]:
combined_img_aug = np.concatenate((HII_Images_0, HII_Images_90, HII_Images_180, HII_Images_270, PNE_Images_0, PNE_Images_90, PNE_Images_180, PNE_Images_270, RG_Images_0, RG_Images_90, RG_Images_180, RG_Images_270), axis = 0)
combined_labels_aug = np.concatenate((HII_Labels_0, HII_Labels_90, HII_Labels_180, HII_Labels_270, PNE_Labels_0, PNE_Labels_90, PNE_Labels_180, PNE_Labels_270, RG_Labels_0, RG_Labels_90, RG_Labels_180, RG_Labels_270), axis = 0)

combined_img_aug.shape

In [ ]:
# Binarising the labels for the CNN:
lb = LabelBinarizer()
combined_labels_aug = lb.fit_transform(combined_labels_aug)

In [ ]:
def cnnFn(combinedImages, combinedLabels, layers: int, neurons: float, neuronsDescending: bool(), numEpochs: int, learnRate: float, momentum: float):
  '''
  layers = number of layers to add

  neurons = integer from 0.5 to 2.0. Either halving or doubling the number of neurons. 

  numEpochs = number of epochs to train the model. 
  '''
  
  trainData, testData, trainLabels, testLabels = train_test_split(combinedImages, combinedLabels, test_size=0.3)

  modelCNN = Sequential()
  size = trainData.shape[1]
  inputShape = (size, size, 3)

  chanDim = -1
  if K.image_data_format() == "channels_first":
    inputShape = (size, size, 3)
    chanDim = 1
  
  neurons = [neurons, neurons]

  modelCNN.add(Conv2D(32*neurons[0], (3,3), input_shape = inputShape))
  modelCNN.add(Activation('relu'))
  modelCNN.add(MaxPooling2D(pool_size=(2,2)))

  modelCNN.add(Conv2D(32*neurons[0], (3,3)))
  modelCNN.add(Activation('relu'))
  modelCNN.add(MaxPooling2D(pool_size=(2,2)))

  if neuronsDescending == True:
    neurons = [neurons[0]*0.5, neurons[1]*0.25]

  modelCNN.add(Conv2D(64*neurons[0], (3,3))) 
  modelCNN.add(Activation('relu'))
  modelCNN.add(MaxPooling2D(pool_size=(2,2)))

  if layers == 1:
    modelCNN.add(Conv2D(64*neurons[1], (3,3))) 
    modelCNN.add(Activation('relu'))
    modelCNN.add(MaxPooling2D(pool_size=(2,2)))

  if layers == 2:
    modelCNN.add(Conv2D(64*neurons[1], (3,3))) 
    modelCNN.add(Activation('relu'))
    modelCNN.add(MaxPooling2D(pool_size=(2,2)))

  modelCNN.add(Flatten()) 
  modelCNN.add(Dense(64))
  modelCNN.add(Activation('relu'))
  modelCNN.add(Dropout(0.5))

  nClasses = 3
  modelCNN.add(Dense(nClasses))
  modelCNN.add(Activation('softmax'))

  # Optimiser and compiler
  opt = SGD(learning_rate= learnRate, momentum= momentum)

  modelCNN.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  batchSize = 32

  # Running the CNN model
  H = modelCNN.fit(trainData, trainLabels, 
                validation_data=(testData, testLabels),
                batch_size= batchSize, epochs = numEpochs, verbose= 0)


  # Plot the train/valid loss and train/valid accuracy 
  plot_train_curves(H.history)

  # Modelling all the test data
  predictionsCNN = modelCNN.predict(testData)
  yPred = predictionsCNN.argmax(axis=1)

  # Printing classification report
  report = classification_report(testLabels.argmax(axis=1),
                                yPred,
                                target_names=["HII","PNE","RG"])
  print(report)

  cmatrix = confusion_matrix(testLabels.argmax(axis=1), yPred)
  plot_confusion_matrix(cmatrix, classes = ['HII', 'PNE', 'RG'],
                        title = 'Classification Confusion Matrix', normalize=True)
  print(cmatrix)

# fastai and pytorch comparison